# Braket Expansion

### Problem Description

Implement a similar behavior to bash's brace expansion behavior as a runnable program.

For a valid input, print the output. For an invalid input, print nothing and exit.

### Considerations

  - Any input without properly matching braces is invalid
  - Commas should only appear within braces. 
  - Restrict the input character set to \[a-zA-Z\{\},\] 
  - Braces should not be empty, and there should be no "empty" options within braces i.e. {A,}

### Examples of Valid Input
  - {A,B,C} -> A B C
  - AB{C,D} -> ABC ABD
  - {A,B}{C,D} -> AC AD BC BD
  - {A,B{C,D}} -> A BC BD
  - {{A},{B}} -> A B
  - {ABC} -> ABC
  - ABC -> ABC

###  Examples of invalid input
  - }ABC
  - {ABC
  - }{
  - {}
  - A,B,C
  - A B C
  - {A{B,C}
  - {A,}


In [1]:
#include <iostream>
#include <sstream>
#include <string>
#include <vector>
#include <stdexcept>
#include <stdio.h>
using namespace std;


In [2]:
typedef vector<string> ExpressionComponents;

class BracketExpression
{
    public:
    
    BracketExpression()
    {
        root = true;
        unmatchedChars = new vector<char>();
    }
    
    BracketExpression(vector<char>*& unmatched): unmatchedChars(unmatched)
    {
    }
    
    ~BracketExpression()
    {
        if(root)
        {
            delete unmatchedChars;
        }
    }
    
    string Expand(const string& inputString);
    
    private:
    
    ExpressionComponents expandedExp;
    ExpressionComponents currentExp;
    vector<char>* unmatchedChars;
    bool root = false;
    const char EXPOPEN = '{';
    const char EXPCLOSE = '}';
    const char EXPSEPARATOR = ',';
    const char EXPTERM = '\0';
    
    ExpressionComponents RecursiveExpand(char *& currentChar);
    void Merge(ExpressionComponents const& mergeExp);

    //Processing functions used for each main component type of the expression syntax 
    void Open(char*& currentChar);
    void Alpha(char*& currentChar);
    ExpressionComponents Close();   
    void Separate();
    
    void SaveExpanded();
    
}

In [3]:
void BracketExpression::Merge(ExpressionComponents const& mergeExp)
{
    ExpressionComponents results;
    if(!mergeExp.empty())
    {
        if(currentExp.empty())
        {
            currentExp = mergeExp;
            return;
        }
        
        for(auto currentComponent : currentExp)
        {
            for(auto mergeComponent : mergeExp)
            {
                results.push_back(currentComponent + mergeComponent);
            }
        }
    }
    else
    {
        return;
    }
    
    currentExp = results;
}

In [4]:
void BracketExpression::SaveExpanded()
{
    if(currentExp.size() > 0)
    {
        std::copy(currentExp.begin(),currentExp.end(), std::back_inserter(expandedExp));
        currentExp.clear();
    }
}

In [5]:
ExpressionComponents BracketExpression::Close()
{
    if(unmatchedChars->empty() || unmatchedChars->back() != EXPCLOSE)
    {
        throw invalid_argument("Invalid input - Close character not expected");
    }
    else
    {
        unmatchedChars->pop_back();
    }

    SaveExpanded();

    return expandedExp;
}

In [6]:
void BracketExpression::Open(char*& currentChar)
{
    if(!unmatchedChars->empty() && unmatchedChars->back() == ',')
    {
        unmatchedChars->pop_back();
    }
            
    unmatchedChars->push_back('}');

    currentChar++;
    
    BracketExpression newExpr(unmatchedChars);
    ExpressionComponents const& returned = newExpr.RecursiveExpand(currentChar);
    Merge(returned);
}

In [7]:
void BracketExpression::Separate()
{
    if(unmatchedChars->empty() || unmatchedChars->back() != '}')
    {
        throw invalid_argument("Invalid input - Separator character not expected");
    } 

    unmatchedChars->push_back(',');

    SaveExpanded();
}

In [8]:
void BracketExpression::Alpha(char*& currentChar)
{
    if(!unmatchedChars->empty() && unmatchedChars->back() == ',')
    {
        unmatchedChars->pop_back();
    }

    Merge(vector<string> {std::string(1,*currentChar)});
}

In [9]:
ExpressionComponents BracketExpression::RecursiveExpand(char*& currentChar)
{   
    while(*currentChar != EXPTERM)
    {
        if(*currentChar == EXPOPEN)
        {   
            Open(currentChar);
        }
        else if(*currentChar == EXPCLOSE)
        {
            return Close();
        }
        else if(*currentChar == EXPSEPARATOR)
        {
            Separate();
        }
        else if(isalpha(*currentChar))
        {
            Alpha(currentChar);
        }
        else
        {
            throw invalid_argument("Invalid input - The character does not fit the syntax of a Bracket Expression");
        }
        
        ++currentChar;
    }
    
    if(!currentExp.empty())
    {
        SaveExpanded();
    }
    
    return expandedExp;
}

In [10]:
string BracketExpression::Expand(const string& inputString)
{
    
    char* currentChar = new char[inputString.length() + 1];
    strcpy(currentChar, inputString.c_str());
    
    vector<string> strings = RecursiveExpand(currentChar);
    
    const char* const separator = " ";
    std::ostringstream imploded;
    std::copy(strings.begin(), strings.end(),
    std::ostream_iterator<std::string>(imploded, separator));
    
    //delete[] currentChar; for some reason this is crashing?
    return imploded.str();
}

In [11]:
const string hello = "{A,B}{{C,D}EF,G}";
BracketExpression brackets;
cout<<brackets.Expand(hello);


ACEF ADEF AG BCEF BDEF BG 